# Query "More Like This" e Classificação

In [1]:
# Bibliotecas:
import pandas as pd
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
import urllib3
import json
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [2]:
import elasticsearch

ES_URL = 'https://localhost:9200'
ES_USER = 'elastic'
ES_PASS = '08win0991i'

client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_USER, ES_PASS),
    verify_certs=False
)

client

C:\Users\winicius.faquieri\Anaconda3\lib\site-packages\elasticsearch\_sync\client\__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


<Elasticsearch(['https://localhost:9200'])>

In [3]:
# Desligando warning para facilitar nossa vida
import urllib3
urllib3.disable_warnings()

In [4]:
client.info()

ObjectApiResponse({'name': 'E012366', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'Sy1H2ZYYQla4vTUqp3-s6A', 'version': {'number': '8.2.2', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '9876968ef3c745186b94fdabd4483e01499224ef', 'build_date': '2022-05-25T15:47:06.259735307Z', 'build_snapshot': False, 'lucene_version': '9.1.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
INDEX_NAME = '20newsgroup_teste_insercao'

# Busca com um documento em string

In [6]:
doc = client.search(index=INDEX_NAME, q='windows', size=150)['hits']['hits'][-1]

In [11]:
print(doc['_source']['message'])

From: paladin@world.std.com (Thomas G Schlatter)
Subject: Re: Tidying up after removing an OLE server
Keywords: OLE, SPSS
Organization: The World Public Access UNIX, Brookline, MA
Lines: 22

In article <C5y121.Kz3@liverpool.ac.uk> qq43@liverpool.ac.uk (Chris Wooff) writes:
>A while ago I installed SPSS for Windows as part of an evaluation. Once
>the evaluation was complete I duly deleted the software from my PC.
>
>Unfortunately there is still a "ghost" of SPSS left: when I run
>something like "Write" and go to embed an object then "SPSS Chart"
>appears on the list of objects I'm offered. I looked around all
>the obvious "INI" files without success. The next thing I tried
>was looking for the string "SPSS Chart" in every file in the 
>Windows directory. It turned up in a file called REQ.DAT (or
>REG.DAT). Unfortunately the file was binary and so I didn't feel
>inclined to edit it.
>
>I'd welcome a solution for removing SPSS from the list of OLE servers.
 I think you can do this with RE

In [13]:
QUERY = {
    'more_like_this': {
        'fields': ['message'],
        'like': [
            {
                '_index': INDEX_NAME,
                '_id': 'F4mTaYEBgNM6tyVNXMNu',
            }
        ],
        'min_term_freq': 1,
        'max_query_terms': 12,
    }
}
resp = client.search(index=INDEX_NAME, query=QUERY, size=10)
resp

ObjectApiResponse({'took': 87, 'timed_out': False, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [9]:
resp_df = pd.DataFrame({'_id': x['_id'], '_score': x['_score'], **x['_source']} for x in resp['hits']['hits'])

In [10]:
resp_df

""


## Classificação baseado somente nos 10 mais próximos

In [43]:
resp_df['newsgroup'].value_counts()

comp.sys.ibm.pc.hardware    3
comp.graphics               2
sci.electronics             2
comp.os.ms-windows.misc     2
soc.religion.christian      1
Name: newsgroup, dtype: int64

In [49]:
resp_df.groupby('newsgroup').sum().index[0]

'comp.graphics'

In [50]:
def classify_with_score(like, size=10):
    
    query = {
        'more_like_this': {
            'fields': ['message'],
            'like': like,
            'min_term_freq': 1,
            'max_query_terms': 12,
        }
    }
    resp = client.search(index=INDEX_NAME, query=query, size=size)
    
    resp_df = pd.DataFrame(
        {
            '_id': x['_id'],
            '_score': x['_score'],
            **x['_source']
        } for x in resp['hits']['hits']
    )
    return resp_df.groupby('newsgroup').sum().index[0]
    

In [14]:
resp_df.groupby('newsgroup').sum().index[0]

KeyError: 'newsgroup'

In [52]:
like = [
    {
        '_index': INDEX_NAME,
        '_id': 'SImTaYEBgNM6tyVNXMNu',
    }
]
classify_with_score(like)

'soc.religion.christian'

In [65]:
def classify_document(doc_id, size=10):
    
    return classify_with_score(
        [
            {
                '_index': INDEX_NAME,
                '_id': doc_id
            }
        ]
    )

In [66]:
classify_document('SImTaYEBgNM6tyVNXMNu')

'soc.religion.christian'

# Avaliando o "classificador"

In [62]:
test_docs_resp = client.search(index=INDEX_NAME, size=100, _source=['newsgroup'])

In [63]:
test_docs = pd.DataFrame(
        {
            '_id': x['_id'],
            **x['_source']
        } for x in test_docs_resp['hits']['hits']
    )

In [64]:
test_docs

,_id,newsgroup
0,Y4mTaYEBgNM6tyVNWLHP,rec.autos
1,ZomTaYEBgNM6tyVNWLHP,comp.graphics
2,bImTaYEBgNM6tyVNWLHP,comp.sys.mac.hardware
3,bomTaYEBgNM6tyVNWLHP,talk.religion.misc
4,cYmTaYEBgNM6tyVNWLHP,misc.forsale
...,...,...
95,SomTaYEBgNM6tyVNWLLQ,comp.os.ms-windows.misc
96,S4mTaYEBgNM6tyVNWLLQ,sci.space
97,TYmTaYEBgNM6tyVNWLLQ,comp.os.ms-windows.misc
98,UYmTaYEBgNM6tyVNWLLQ,rec.sport.baseball


In [67]:
test_docs['predicted'] = test_docs['_id'].apply(classify_document)

In [68]:
test_docs

,_id,newsgroup,predicted
0,Y4mTaYEBgNM6tyVNWLHP,rec.autos,misc.forsale
1,ZomTaYEBgNM6tyVNWLHP,comp.graphics,comp.graphics
2,bImTaYEBgNM6tyVNWLHP,comp.sys.mac.hardware,comp.graphics
3,bomTaYEBgNM6tyVNWLHP,talk.religion.misc,alt.atheism
4,cYmTaYEBgNM6tyVNWLHP,misc.forsale,misc.forsale
...,...,...,...
95,SomTaYEBgNM6tyVNWLLQ,comp.os.ms-windows.misc,comp.os.ms-windows.misc
96,S4mTaYEBgNM6tyVNWLLQ,sci.space,sci.space
97,TYmTaYEBgNM6tyVNWLLQ,comp.os.ms-windows.misc,comp.graphics
98,UYmTaYEBgNM6tyVNWLLQ,rec.sport.baseball,comp.windows.x


In [69]:
from sklearn.metrics import classification_report

In [70]:
print(
    classification_report(
        test_docs['newsgroup'],
        test_docs['predicted']
    )
)

                          precision    recall  f1-score   support

             alt.atheism       0.50      1.00      0.67         2
           comp.graphics       0.39      0.88      0.54         8
 comp.os.ms-windows.misc       0.64      0.70      0.67        10
comp.sys.ibm.pc.hardware       0.25      0.50      0.33         2
   comp.sys.mac.hardware       0.80      0.57      0.67         7
          comp.windows.x       0.33      0.50      0.40         4
            misc.forsale       0.43      0.43      0.43         7
               rec.autos       0.33      0.25      0.29         4
         rec.motorcycles       0.80      0.57      0.67         7
      rec.sport.baseball       1.00      0.40      0.57         5
        rec.sport.hockey       1.00      0.80      0.89         5
               sci.crypt       0.75      1.00      0.86         3
         sci.electronics       0.00      0.00      0.00         3
                 sci.med       1.00      0.40      0.57         5
         

/home/felipefg/.pyenv/versions/3.9.1/envs/infnet/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/felipefg/.pyenv/versions/3.9.1/envs/infnet/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/felipefg/.pyenv/versions/3.9.1/envs/infnet/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_